In [2]:
#importing the modules we need
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
import numpy as np

In [3]:
# the normalize function
def normalize_data(train_data, test_data, type1=None):
    scaler = None

    if type1 == 'standard':
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data
    
    elif type1 == 'l1':
        scaler = preprocessing.Normalizer(norm='l1')
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data
    
    elif type1 == 'l2':
        scaler = preprocessing.Normalizer(norm='l2')
        scaler.fit(train_data)
        
        scaled_train_data = scaler.transform(train_data)
        scaled_test_data = scaler.transform(test_data)
        
        return scaled_train_data, scaled_test_data

In [4]:
#bag of words class, made during ml lab
class BagOfWords:
    def __init__(self):
        self.vocabulary = dict()
        self.words = []  
        
        
    def build_vocabulary(self, data):
        for sentence in data:
            for word in sentence:
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
                    self.words.append(word)
                    
            
    def get_features(self, data):
        features = np.zeros((len(data), len(self.vocabulary)))
        
        for id_sen, document in enumerate(data):
            for word in document:
                if word in self.vocabulary:
                    features[id_sen, self.vocabulary[word]] += 1
                    
        return features

In [5]:
#getting the data
def load_sample(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    indexes = []
    sentences = []
    
    for line in f.readlines():
        indexes.append(int("".join(line[:6])))
        sentences.append(line[7:].strip('\n').split())
        
    return indexes, sentences


def load_label(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    sentences = []
    
    for line in f.readlines():
        sentences.append(int(line[7]))
        
    return sentences

In [6]:
#train data
train_indexes, train_samples = load_sample("data/train_samples.txt")
train_labels = load_label("data/train_labels.txt")

#validation data
validation_indexes, validation_samples = load_sample("data/validation_samples.txt")
validation_labels = load_label("data/validation_labels.txt")

#test data
test_indexes, test_samples = load_sample("data/test_samples.txt")

In [7]:
bow = BagOfWords()
bow.build_vocabulary(train_samples)

train_features = bow.get_features(train_samples)
validation_features = bow.get_features(validation_samples)
test_features = bow.get_features(test_samples)


In [9]:
print(train_features[0])

[1. 1. 2. ... 0. 0. 0.]


In [16]:
#train the model
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(train_features, train_labels)

MultinomialNB()

In [17]:
#get the prediction on the validation data
predicted = naive_bayes_model.predict(validation_features)

print(np.mean(predicted == validation_labels))

0.7304


In [20]:
train_features = bow.get_features(train_samples + validation_samples)

#train the final model
naive_bayes_model_final = MultinomialNB()
naive_bayes_model_final.fit(train_features, train_labels + validation_labels)

#get the prediction on the test data
predicted = naive_bayes_model_final.predict(test_features)

#and write it in the csv
g = open("data/test_labels.txt", 'w')
g.write("id,label\n")

for idx in range(len(predicted)):
    g.write(f"{test_indexes[idx]},{predicted[idx]}\n")